In [95]:
# 加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [96]:
# 定义超参数
'''
参数：模型f（x，a）中的a称为模型的参数，可以通过优化算法进行学习
超参数：定义模型结构或优化策略
'''
BATCH_SIZE = 64 # 若电脑性能不好可以减小
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 10 # 训练数据集的轮次

In [97]:
# 构建pipeline（transforms），对图像做处理
pipeline = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换成tensor
    transforms.Normalize((0.1307,),(0.3081,)) # 归一化
])

In [98]:
# 下载加载数据
from torch.utils.data import DataLoader

train_set = datasets.MNIST("data", train=True, download=True, transform=pipeline)
test_set = datasets.MNIST("data", train=False, download=True, transform=pipeline)

# 加载数据
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) # 打乱
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True) # 打乱


In [99]:
# 这段用于查看数据集内的图片
# with open("./data/MNIST/raw/t10k-images-idx3-ubyte", "rb") as f:
#     file = f.read()
# imagel = [int(str(item).encode("ascii"), 16) for item in file[16: 16+784]]
# print(imagel)
# import cv2
# import numpy as np
# imagel_np = np.array(imagel, dtype=np.int8).reshape(28, 28, 1)
# cv2.imwrite("digit.jpg", imagel_np)

In [100]:
# 构建网络模型
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, 5) # 1: 灰度图片的通道 10：输出通道 5：kernel
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.fc1 = nn.Linear(20*10*10, 500) # 20*10*10:输入通道  500：输出通道
        self.fc2 = nn.Linear(500, 10) # 500: 输入通道， 10：输出通道

    def forward(self, x):
        input_size = x.size(0)  # batch_size*1*28*28
        x = self.conv1(x) # 输入：batch_size*1*28*28， 输出：batch_size*10*24*24  （28 - 5 + 1）= 24
        x = F.relu(x) # 激活函数 保持shape不变，输出：batch_size*10*24*24
        x = F.max_pool2d(x, 2, 2) # 输入：batch_size*10*24*24 输出：batch_size*10*12*12

        x = self.conv2(x) #输入：batch_size*10*12*12 输出：：batch_size*20*10*10         （12 - 3 + 1）= 10
        x = F.relu(x)

        x = x.view(input_size, -1) # 展开（拉平） -1 自动计算维度，20*10*10  输出：batch_size*20*10*10 = batch_size*2000

        x = self.fc1(x) # 输入:batch_size*2000 输出：batch*500
        x = F.relu(x) # 保持shape不变

        x = self.fc2(x) # 输入：batch*500 输出：batch*10

        output = F.log_softmax(x, dim=1) # 计算分类后每个数字的概率值

        return output


In [101]:
# 定义优化器
model = Digit().to(DEVICE)

optimizer = optim.Adam(model.parameters())

In [102]:
# 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    # 模型训练
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        # 部署到DEVICE上
        data, target = data.to(device), target.to(device)
        # 梯度初始化为0
        optimizer.zero_grad()
        # 预测
        output = model(data)
        # 计算损失
        loss = F.cross_entropy(output, target)  # 交叉熵函数
        # 找到概率最大的下标
        # pred = output.max(1, keepdim=True) # pred = output.argmax(dim=1)
        # 反向传播
        loss.backward()
        # 参数优化
        optimizer.step()
        if batch_index % 3000 == 0: 
            print(f"Train Epoch:{epoch} \t Loss:{loss.item():.6f}\n")


In [103]:
# 测试方法：
def test_model(model, device,test_loader):
    # 模型验证
    model.eval()
    # 正确率
    currect = 0.0
    # 测试损失
    test_loss = 0.0
    with torch.no_grad():   # 不会计算梯度，也不会进行反向传播
        # 部署到device上
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            # 测试数据
            output = model(data)
            # 计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            # 找到概率最大的下标
            pred = output.max(1, keepdim=True)[1]  # 值， 索引
            # pred = torch.max(output, dim=1)
            # pred = output.argmax(dim=1)
            # 累计正确的值
            currect += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print(f"Test —— Average loss : {test_loss:.4f}, Accuracy : { 100.0*currect / len(test_loader.dataset):.3f}\n")



In [104]:
# 调用方法
for epoch in range(1, EPOCHS + 1):
    train_model(model, DEVICE, train_loader, optimizer, epoch)
    test_model(model, DEVICE, test_loader)

Train Epoch:1 	 Loss:2.294555

Test —— Average loss : 0.0007, Accuracy : 98.360

Train Epoch:2 	 Loss:0.015559

Test —— Average loss : 0.0005, Accuracy : 98.850

Train Epoch:3 	 Loss:0.006279

Test —— Average loss : 0.0005, Accuracy : 98.770

Train Epoch:4 	 Loss:0.003055

Test —— Average loss : 0.0007, Accuracy : 98.660

Train Epoch:5 	 Loss:0.004222

Test —— Average loss : 0.0005, Accuracy : 99.010

Train Epoch:6 	 Loss:0.006399

Test —— Average loss : 0.0007, Accuracy : 98.810

Train Epoch:7 	 Loss:0.000569

Test —— Average loss : 0.0006, Accuracy : 98.920

Train Epoch:8 	 Loss:0.000056

Test —— Average loss : 0.0006, Accuracy : 98.950

Train Epoch:9 	 Loss:0.000147

Test —— Average loss : 0.0006, Accuracy : 98.930

Train Epoch:10 	 Loss:0.000314

Test —— Average loss : 0.0011, Accuracy : 98.330

